GPT

In [1]:
from openai import AsyncOpenAI, OpenAI
from anthropic import AsyncAnthropic
from google.genai import types
from google import genai
from mistralai import Mistral

In [10]:
from dotenv import load_dotenv
import os, base64
from glom import glom
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

load_dotenv()


True

In [27]:
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
def encode_image(image_path : str):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [6]:
image_path = "C:\\test_test\\response_rate_of_image_description\\data\\table_test.png"
    
base64_image = encode_image(image_path)

class ImageDescription(BaseModel):
    category : str = Field(description = "이미지의 카테고리. 4가지 카테고리 : ['figure', 'chart', 'table', 'equation']")
    description : str = Field(description = "Describe image in detail")

In [28]:
@tool
async def gpt_vision_tool(image_path : str):
    """
    이미지 경로를 통해서 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하는 툴
    """

    base64_image = encode_image(image_path)

    ext = os.path.splitext(image_path)[1]

    if "." in ext:
        print(f". 있어요. {ext}")
        ext = ext.replace('.', '')

    message_list = [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": """주어진 이미지를 분석하세요. 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하세요. 
                    표의 경우 담겨있는 정보를 생략하지 말고 전부 일목요연하게 제공하세요. 설명의 경우 마크다운으로 작성하세요(```markdown 은 사용하지 말고 마크다운 형식으로 작성하세요)
                    수식의 경우 KaTex 포맷으로 작성하세요. $$ $$ or $ $ 형식으로 작성하세요
                    """
                },
                {
                    "type": "input_image",
                    "image_url": f"data:image/{ext};base64,{base64_image}",
                    "detail": "high"
                }
            ]
        }
    ]

    

    response = await client.responses.parse(
        model = "gpt-4.1",
        input = message_list,
        text_format = ImageDescription
    )

    print(response)
    return response
    

In [11]:
async def get_image_description_agent(image_path : str, llm):

    system_message = SystemMessage(
    content = f"""
    image_path {image_path}

    당신은 제공받은 이미지 경로를 이용하여 이미지 카테고리('figure', 'chart', 'table', 'equation')과 설명을 이용하여 이미지를 분석하는 봇입니다.

    gpt_vision_tool 툴을 이용하여 이미지 카테고리와 이미지 설명을 반환받으세요

    제공받은 설명을 완벽한 마크다운 형태로 리포맷팅(```markdown 태그는 사용하지 말고 마크다운 형식으로 작성하세요) 하세요.
    """
    )

    tools = [gpt_vision_tool]

    agent = create_react_agent(
        llm,
        tools,
        prompt = system_message
    )

    return agent

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph

In [14]:

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
agent = await get_image_description_agent(image_path, llm)

In [21]:
inputs = {
    "messages" : [
        ("user", "주어진 이미지를 분석하세요")
    ]
}

config = RunnableConfig(
    recursion_limit=100
)

In [29]:
async for chunk_msg, metadata in agent.astream(inputs, config , stream_mode="messages"):
    curr_node = metadata['langgraph_node']
    print(f"Node : {curr_node}")
    print(f"chunk_msg : {chunk_msg}")
    print(f"metadata : {metadata}")

Node : agent
chunk_msg : content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_yZDDRB1d65cVdUf3xiipZe62', 'function': {'arguments': '', 'name': 'gpt_vision_tool'}, 'type': 'function'}]} response_metadata={} id='run--80b0c910-20eb-4c36-a19a-2fc28db42d16' tool_calls=[{'name': 'gpt_vision_tool', 'args': {}, 'id': 'call_yZDDRB1d65cVdUf3xiipZe62', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'gpt_vision_tool', 'args': '', 'id': 'call_yZDDRB1d65cVdUf3xiipZe62', 'index': 0, 'type': 'tool_call_chunk'}]
metadata : {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:f947a6ec-6259-d9e1-6024-ac303c41c1e6', 'checkpoint_ns': 'agent:f947a6ec-6259-d9e1-6024-ac303c41c1e6', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1', 'ls_model_type': 'chat', 'ls_temperature': 0.0}
Node : agent
chunk_msg : content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None

c:\Users\COM-NMOIM\miniforge3\envs\graph-module\Lib\typing.py:1696: RuntimeWarning: coroutine 'AsyncResponses.parse' was never awaited
  return hash(frozenset(self.__args__))


Node : agent
chunk_msg : content='' additional_kwargs={} response_metadata={} id='run--45563496-72cc-49d3-9d03-5347d94b7dba'
metadata : {'langgraph_step': 3, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:9e928071-59d0-4c11-1ef8-121e35c74e31', 'checkpoint_ns': 'agent:9e928071-59d0-4c11-1ef8-121e35c74e31', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1', 'ls_model_type': 'chat', 'ls_temperature': 0.0}
Node : agent
chunk_msg : content='이미' additional_kwargs={} response_metadata={} id='run--45563496-72cc-49d3-9d03-5347d94b7dba'
metadata : {'langgraph_step': 3, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:9e928071-59d0-4c11-1ef8-121e35c74e31', 'checkpoint_ns': 'agent:9e928071-59d0-4c11-1ef8-121e35c74e31', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1', 'ls_model_type': 'chat